# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

from itertools import chain, combinations

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(filepath,'*'))
    print(file_path_list)

/home/workspace
['/home/workspace/event_data/2018-11-27-events.csv', '/home/workspace/event_data/2018-11-04-events.csv', '/home/workspace/event_data/2018-11-07-events.csv', '/home/workspace/event_data/2018-11-09-events.csv', '/home/workspace/event_data/2018-11-19-events.csv', '/home/workspace/event_data/2018-11-05-events.csv', '/home/workspace/event_data/2018-11-22-events.csv', '/home/workspace/event_data/2018-11-16-events.csv', '/home/workspace/event_data/2018-11-26-events.csv', '/home/workspace/event_data/2018-11-24-events.csv', '/home/workspace/event_data/2018-11-29-events.csv', '/home/workspace/event_data/2018-11-15-events.csv', '/home/workspace/event_data/2018-11-20-events.csv', '/home/workspace/event_data/2018-11-06-events.csv', '/home/workspace/event_data/2018-11-18-events.csv', '/home/workspace/event_data/2018-11-21-events.csv', '/home/workspace/event_data/2018-11-10-events.csv', '/home/workspace/event_data/2018-11-23-events.csv', '/home/workspace/event_data/2018-11-02-events.c

#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [37]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


8056


In [4]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


In [5]:
from itertools import chain, combinations
combination_list = []
df = pd.read_csv('event_datafile_new.csv')

def key_options(items):
    
    """
    Get's every combination of Primary Keys and appends them to combination_list
    
    Arguments:
        * items: lists of Dataframe Keys
    Returns:
        Get's every combination of Primary Keys
    """
    
    return chain.from_iterable(combinations(items, r) for r in range(1, len(items)+1) )

for candidate in key_options(list(df)):
    deduped = df.drop_duplicates(candidate)
    


    if len(deduped.index) == len(df.index):
        combination_list.append(','.join(candidate))

In [6]:
#combination_list


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [7]:
# Connection to a Cassandra instance your local machine 
# (127.0.0.1)


from cassandra.cluster import Cluster

try:
    cluster = Cluster(['127.0.0.1'])

# To establish connection and begin executing queries, need a session
    session = cluster.connect()
except Exception as e:
    print('Error: error creating cluster or session')
    print(e)

#### Create Keyspace

In [8]:
## Create Keyspace for sparkify Cassanda DB
try:
    session.execute("""
    Create KEYSPACE IF NOT EXISTS sparkify
    WITH REPLICATION =
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 } """)
except Exception as e:
    print('Error : Issue Creating Keyspace')
    print(e)

#### Set Keyspace

In [9]:
## Set KEYSPACE to the keyspace specified above
try:
    session.set_keyspace('sparkify')
except Exception as e:
    print('Error: Error setting Keyspace')
    print(e)


### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




In [10]:
list(df)

['artist',
 'firstName',
 'gender',
 'itemInSession',
 'lastName',
 'length',
 'level',
 'location',
 'sessionId',
 'song',
 'userId']

In [11]:
#df.sessionId.unique()

In [12]:
df[(df['sessionId'] == 338) & (df['itemInSession'] == 4.0)]

,artist,firstName,gender,itemInSession,lastName,length,level,location,sessionId,song,userId
444,Faithless,Ava,F,4,Robinson,495.3073,free,"New Haven-Milford, CT",338,Music Matters (Mark Knight Dub),50


In [13]:
#df.groupby(['sessionId', 'itemInSession']).count()

In [14]:
if 'itemInSession,sessionId' in combination_list:
    print('yes')
else:
    print('No')

yes


In [39]:
df[['artist', 'song', 'length']].describe()

,length
count,6820.000000
mean,247.032221
std,102.975921
min,15.855870
25%,197.321998
50%,232.972605
75%,274.121992
max,2594.873020


In [15]:
## Query 1:  Give me the artist, song title and song's length in the music app history that was heard during \
## sessionId = 338, and itemInSession = 4

## Create Table called song_sessions to answer query 1

# 
# session_id and item_in_session is a Composite Primary Key
# A Composite PRIMARY KEY is made up of more than one column and will assist in creating a unique value and in your retrieval queries
# The Composite PRIMARY KEY will determine the distribution of data across the system


query = "CREATE TABLE IF NOT EXISTS song_sessions "

query = query + "(artist Text, \
                  song_title Text, \
                  song_length decimal, \
                  session_id int, \
                  item_in_session int, \
                  PRIMARY KEY (session_id, item_in_session))"

try:
    session.execute(query)
except Exception as e:
    print('Error: Creating sessions table')
    print(e)


                    

In [16]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6820 entries, 0 to 6819
Data columns (total 11 columns):
artist           6820 non-null object
firstName        6820 non-null object
gender           6820 non-null object
itemInSession    6820 non-null int64
lastName         6820 non-null object
length           6820 non-null float64
level            6820 non-null object
location         6820 non-null object
sessionId        6820 non-null int64
song             6820 non-null object
userId           6820 non-null int64
dtypes: float64(1), int64(3), object(7)
memory usage: 586.2+ KB


In [17]:
# We have provided part of the code to set up the CSV file. Please complete the Apache Cassandra code below#
## Opens event_datafile_new CSV and reads each line and inserts data into song_sessions table

file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        
## TO-DO: Assign the INSERT statements into the `query` variable

        query = "INSERT INTO song_sessions \
                 (artist, \
                  song_title, \
                  song_length, \
                  session_id, \
                  item_in_session)"
    
        query = query + "VALUES (%s, %s, %s, %s, %s)"
        
        ## TO-DO: Assign which column element should be assigned for each column in the INSERT statement.
        ## For e.g., to INSERT artist_name and user first_name, you would change the code below to `line[0], line[1]`
        session.execute(query, (line[0], line[9], float(line[5]), int(line[8]), int(line[3])))

#### Do a SELECT to verify that the data have been inserted into each table

In [18]:
## Add in the SELECT statement to verify the data was entered into the table

## Runs Select Statement to check if query 1 was answered correctly

try:
    rows = session.execute('SELECT * FROM song_sessions where session_id = 338 and item_in_session = 4')
    
except Exception as e:
    print('Error: Select Statement')
    print(e)

for row in rows:
    print(row.artist, row.song_title, row.song_length)
    

Faithless Music Matters (Mark Knight Dub) 495.3073


### COPY AND REPEAT THE ABOVE THREE CELLS FOR EACH OF THE THREE QUESTIONS

In [19]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6820 entries, 0 to 6819
Data columns (total 11 columns):
artist           6820 non-null object
firstName        6820 non-null object
gender           6820 non-null object
itemInSession    6820 non-null int64
lastName         6820 non-null object
length           6820 non-null float64
level            6820 non-null object
location         6820 non-null object
sessionId        6820 non-null int64
song             6820 non-null object
userId           6820 non-null int64
dtypes: float64(1), int64(3), object(7)
memory usage: 586.2+ KB


In [20]:
#combination_list

In [21]:
if 'itemInSession,sessionId,userId' in combination_list:
    print('yes')
else:
    print('No')

yes


In [22]:
df[(df['userId'] == 10) & (df['sessionId'] == 182.0)]

,artist,firstName,gender,itemInSession,lastName,length,level,location,sessionId,song,userId
4704,Down To The Bone,Sylvie,F,0,Cruz,333.76608,free,"Washington-Arlington-Alexandria, DC-VA-MD-WV",182,Keep On Keepin' On,10
4705,Three Drives,Sylvie,F,1,Cruz,411.63710,free,"Washington-Arlington-Alexandria, DC-VA-MD-WV",182,Greece 2000,10
4706,Sebastien Tellier,Sylvie,F,2,Cruz,377.73016,free,"Washington-Arlington-Alexandria, DC-VA-MD-WV",182,Kilometer,10
4707,Lonnie Gordon,Sylvie,F,3,Cruz,181.21098,free,"Washington-Arlington-Alexandria, DC-VA-MD-WV",182,Catch You Baby (Steve Pitron & Max Sanna Radio...,10


In [40]:
df[['artist', 'song', 'firstName', 'lastName', 'itemInSession', 'userId', 'sessionId']].describe()

,itemInSession,userId,sessionId
count,6820.000000,6820.000000,6820.000000
mean,22.761144,54.681232,599.181818
std,23.444636,28.162734,284.953333
min,0.000000,2.000000,3.000000
25%,4.000000,29.000000,374.000000
50%,15.000000,49.000000,605.000000
75%,35.000000,80.000000,834.000000
max,126.000000,101.000000,1114.000000


In [23]:
## Query 2: Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name)\
## for userid = 10, sessionid = 182

# https://docs.datastax.com/en/dse/5.1/cql/cql/cql_using/whereClustering.html
# Clustering Columns: order data within a partition
# A Composite PRIMARY KEY is made up of more than one column and will assist in creating a unique value and in your retrieval queries
# The Composite PRIMARY KEY will determine the distribution of data across the system
# user_id and session_id are a Composite Partition Key
# item_in_session is a Clustering Column used to order in Ascending Order
# Primary Key is a combination of the Composite Partition Key and Clustering Column ((user_id, session_id), item_in_session)


## CREATES TABLE music_library to answer

query = 'CREATE TABLE IF NOT EXISTS user_artist_session'

query = query + '(artist Text, \
                  song_title Text, \
                  first_name Text , \
                  last_name Text, \
                  item_in_session int, \
                  user_id int , \
                  session_id int , \
                  PRIMARY KEY((user_id, session_id), item_in_session)) \
                  WITH CLUSTERING ORDER BY (item_in_session ASC)'

try:
    session.execute(query)
except Exception as e:
    print('Error: Creating music library table')
    print(e)


                    

In [24]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6820 entries, 0 to 6819
Data columns (total 11 columns):
artist           6820 non-null object
firstName        6820 non-null object
gender           6820 non-null object
itemInSession    6820 non-null int64
lastName         6820 non-null object
length           6820 non-null float64
level            6820 non-null object
location         6820 non-null object
sessionId        6820 non-null int64
song             6820 non-null object
userId           6820 non-null int64
dtypes: float64(1), int64(3), object(7)
memory usage: 586.2+ KB


In [25]:
## Opens event_datafile_new CSV and reads each line and inserts data into sessions table

file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        
## TO-DO: Assign the INSERT statements into the `query` variable


        query = "INSERT INTO user_artist_session (artist, \
                                            song_title, \
                                            first_name, \
                                            last_name, \
                                            item_in_session, \
                                            user_id, \
                                            session_id)"
    
        query = query + "VALUES (%s, %s, %s, %s, %s, %s, %s)"
        
        ## TO-DO: Assign which column elements for insert variable[column index number].
       
        session.execute(query, (line[0], line[9], line[1], line[4], int(line[3]), int(line[10]), int(line[8])))

In [26]:
## Select query to check if question 2 was answered correctly


query = 'SELECT * FROM user_artist_session where user_id = 10 and session_id = 182'

try:
    rows = session.execute(query)
except Exception as e:
    print('Error: SELECT FROM user_artist_session')
    print(e)
    
for row in rows:
    print(row.artist, row.song_title, row.first_name, row.last_name)

Down To The Bone Keep On Keepin' On Sylvie Cruz
Three Drives Greece 2000 Sylvie Cruz
Sebastien Tellier Kilometer Sylvie Cruz
Lonnie Gordon Catch You Baby (Steve Pitron & Max Sanna Radio Edit) Sylvie Cruz


3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

In [27]:

query_three_df = df[['artist', 'firstName', 'lastName', 'song', 'userId']]

query_three_df[query_three_df.song == "All Hands Against His Own"]

,artist,firstName,lastName,song,userId
2792,The Black Keys,Tegan,Levine,All Hands Against His Own,80
5135,The Black Keys,Sara,Johnson,All Hands Against His Own,95
6298,The Black Keys,Jacqueline,Lynch,All Hands Against His Own,29


In [28]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6820 entries, 0 to 6819
Data columns (total 11 columns):
artist           6820 non-null object
firstName        6820 non-null object
gender           6820 non-null object
itemInSession    6820 non-null int64
lastName         6820 non-null object
length           6820 non-null float64
level            6820 non-null object
location         6820 non-null object
sessionId        6820 non-null int64
song             6820 non-null object
userId           6820 non-null int64
dtypes: float64(1), int64(3), object(7)
memory usage: 586.2+ KB


In [29]:
if 'artist' in combination_list:
    print('yes')
else:
    print('No')

No


In [41]:
df[['firstName', 'lastName', 'song', 'userId']].describe()

,userId
count,6820.000000
mean,54.681232
std,28.162734
min,2.000000
25%,29.000000
50%,49.000000
75%,80.000000
max,101.000000


In [30]:
## Query 3: Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

## Create table user_listen_history if not exists to answer question 3

# A Composite PRIMARY KEY is made up of more than one column and will assist in creating a unique value and in your retrieval queries
# The Composite PRIMARY KEY will determine the distribution of data across the system
# song_title and user_id are a Composite Partition Key making up the Primary Key


query = "CREATE TABLE IF NOT EXISTS user_listen_history"

query = query + "(first_name Text, \
                  last_name Text, \
                  song_title Text, \
                  user_id int, \
                  PRIMARY KEY (song_title, user_id))"

try:
    session.execute(query)
except Exception as e:
    print('Error: Error creating user_listen_history table')
    print(e)


In [31]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6820 entries, 0 to 6819
Data columns (total 11 columns):
artist           6820 non-null object
firstName        6820 non-null object
gender           6820 non-null object
itemInSession    6820 non-null int64
lastName         6820 non-null object
length           6820 non-null float64
level            6820 non-null object
location         6820 non-null object
sessionId        6820 non-null int64
song             6820 non-null object
userId           6820 non-null int64
dtypes: float64(1), int64(3), object(7)
memory usage: 586.2+ KB


In [32]:
## Opens event_datafile_new CSV and reads each line and inserts data into sessions table

file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        
## TO-DO: Assign the INSERT statements into the `query` variable

        query = 'INSERT INTO user_listen_history (first_name, \
                                                  last_name, \
                                                  song_title, \
                                                  user_id)'
    
        query = query + ' VALUES (%s, %s, %s, %s)'
        
        ## TO-DO: Assign which column elements for insert variable[column index number].
       
        session.execute(query, (line[1], line[4], line[9], int(line[10])))

In [33]:
## Select statement to check if question 3 is answered correctly

query = "SELECT * FROM user_listen_history where song_title = 'All Hands Against His Own'"

try:
    rows = session.execute(query)
except Exception as e:
    print('Error: Selecting user_listen history')
    print(e)

for row in rows:
    print(row.first_name, row.last_name)

Jacqueline Lynch
Tegan Levine
Sara Johnson


### Drop the tables before closing out the sessions

In [34]:
## Drop the table before closing out the sessions

drop_query1 = "DROP TABLE IF EXISTS sessions; "
drop_query2 = "DROP TABLE IF EXISTS music_library"
drop_query3 = "DROP TABLE IF EXISTS user_listen_history"

try:
    session.execute(drop_query1)
except Execption as e:
    print('Error: dropping sessions')
    print(e)

try:
    session.execute(drop_query2)
except Exception as e:
    print('Error: Dropping music_library')
    print(e)

try:
    session.execute(drop_query3)
except Exception as e:
    print('Error: Dropping user_listen_history')
    print(e)

### Close the session and cluster connection¶

In [35]:
session.shutdown()
cluster.shutdown()